## Initialization
Read global config file, all parameters are supposed to be defined here.

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import yaml

config_path = '/app/global_config/pod_data_prep_v0.yaml'
with open(config_path, 'r') as stream:
    configs = yaml.load(stream,  Loader=yaml.FullLoader)
    
print('Loaded config:')
print(yaml.dump(configs))#  matplotlib inline

### Init nuscenes dataloader

In [ ]:
import os
import json
import numpy as np

def detection_rate(tp_,fp_,fn_):
    return 0 if tp_ == 0 else tp_/(tp_+fn_)


def true_positive_rate(tp_,fp_,fn_):
    return 0 if tp_ == 0 else tp_/(tp_+fp_) 

def calc_response_from_binary(  sortind: int,
                                tp: list,
                                fp: list,
                                fn: list,
                                x_val: list,
                                response_fct: callable,
                                plot_res:int,
                                ):
    assert(len(tp)==len(fp)==len(fn)==len(x_val)==len(x_val))
    if sum(tp) == 0:
        return 0,0

    tp = [tp[i] for i in sortind]
    fp = [fp[i] for i in sortind]
    fn = [fn[i] for i in sortind]
    x_val = [x_val[i] for i in sortind]

    
    n = int(plot_res)
    x_val_ = []
    response_ = []
    for i in range(0, len(tp), n):
        tp_ = sum(tp[i:i + n])
        fp_ = sum(fp[i:i + n])
        fn_ = sum(fn[i:i + n])
        
        response_val = response_fct(tp_,fp_,fn_)
        response_val = np.finfo('float').eps if response_val == 0 else response_val

        # return is not allowed to be 0 or inf or -inf
        # use eps instead
        #assert(not response_val == np.inf and response_val >= 0)
        assert(0 <= response_val < np.inf)

        response_.append(response_val)
        x_val_.append(np.mean(x_val[i:i + n]))

    return x_val_, response_

path = configs['detection_candidate_fusion']['resultpath']
name = 'results.json'

plot_res = 1000

response = true_positive_rate
response_postfix = 'true_positive_rate'
process_parameter = 'score'

#response = detection_rate
#response_postfix = 'detection_rate'
#process_parameter = 'dist'

data = {}
for root, dirs, files in os.walk(path):
    if name in files:
        filepath = os.path.join(root, name)
        str_1 = filepath.split('/')[-3]
        str_2 = filepath.split('/')[-2]

        with open(os.path.join(root, name), 'r') as f:
            data_ = json.load(f)
        
        # check if data quantity is sufficient
        min_points = plot_res*100
        act_points = len(data_['tp'])
        if act_points <= min_points:
            print(f'Case {str_1} {str_2} has only {act_points} datapoints, min of {min_points} is expected')

        ## calculate signal response
        sortind = [i for (v, i) in sorted((v, i) for (i, v) in enumerate(data_[process_parameter]))][::-1]
        sortind = [i for i in sortind if data_[process_parameter][i] > 0]
        data_[process_parameter + '_'],data_[response_postfix] = calc_response_from_binary( sortind,
                                                                    data_['tp'],
                                                                    data_['fp'],
                                                                    data_['fn'],
                                                                    data_[process_parameter],
                                                                    plot_res = plot_res,
                                                                    response_fct=response)

        data[(str_1,str_2)] = data_
        
data.keys()

## plot raw data

In [ ]:
from matplotlib import pyplot as plt
from pod.plot import plot_pod, plot_pod_logx, plot_pod_logy, plot_pod_logx_logy

key = ('CenterPoint_Voxel_01', 'pedestrian')
v = data[key]

y = np.asarray(v[response_postfix])
x = np.asarray(v[process_parameter + '_']) 

fig, ax = plt.subplots(2, 2, figsize=(14, 14))
fig.suptitle(f'{key[0]} - {key[1]}')

plot_pod(ax[0][0],x,y,process_parameter + ' [m]',response_postfix + ' [%]')
plot_pod_logx(ax[1][0],x,y,process_parameter + ' [m]',response_postfix + ' [%]')
plot_pod_logy(ax[0][1],x,y,process_parameter + ' [m]',response_postfix + ' [%]')
plot_pod_logx_logy(ax[1][1],x,y,process_parameter + ' [m]',response_postfix + ' [%]')

In [ ]:
from pod.plot import plot_pod, plot_pod_logx, plot_pod_logy, plot_pod_logx_logy

def linearize_data_distribution(x,y,print_fig = None,destination = True, x_label = 'process parameter',y_label = 'response'):
    fig, ax = plt.subplots(2, 2, figsize=(14, 14))
    fig.suptitle(f'{key[0]} - {key[1]} \n ')
    
    coef = {}
    coef['x'] = x.tolist()
    coef['y'] = y.tolist()
    coef['x-y'] = plot_pod(ax[0][0],x,y,x_label,y_label)
    coef['logx-y'] = plot_pod_logx(ax[1][0],x,y,x_label,y_label)
    coef['x-logy'] = plot_pod_logy(ax[0][1],x,y,x_label,y_label)
    coef['logx-logy'] = plot_pod_logx_logy(ax[1][1],x,y,x_label,y_label)

    # safe results
    if destination:
        if not os.path.exists(destination):
            os.makedirs(destination)

        if print_fig:
            filename = '_'.join(key) + '.png'
            fig_path = os.path.join(destination,'..',filename)    
            fig.savefig(fig_path, dpi=fig.dpi)

        result_path = os.path.join(destination,'coef.json')
        with open(result_path, 'w') as f:
            json.dump(coef, f)
            
    return coef

coef = {}
for key,v in data.items():
    y = np.asarray(v['recall'])
    x = np.asarray(v['dist_']) 
    
    print(f'{type(x.tolist())}, {type(y.tolist())}')
    print(f'{len(x.tolist())}, {len(y.tolist())}')


    destination = configs['detection_candidate_fusion']['resultpath']
    destination = os.path.join(destination,response_postfix,key[0],key[1])
    
    coef[key] = linearize_data_distribution(x,y,
                                            print_fig = True,
                                            destination = destination,
                                            x_label = 'distance [m]',
                                            y_label = 'recall [%]')
